In [1]:
import tensorwrap as tf
tf.__version__, tf.config.set_visible_devices("cuda"), tf.config.list_physical_devices("GPU")

('0.0.0.4',
 None,
 [StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)])

In [2]:
from tensorwrap import nn

In [3]:
layer = tf.nn.layers.Dense(units = 10)
layer1 = tf.nn.layers.Dense(units = 1)

In [4]:
x = tf.range(1, 1e7, dtype=tf.float32)
x = tf.expand_dims(x, axis = 1)
y = x + 10
x /= 1e5

In [5]:
model = nn.Sequential([
    layer,
    layer,
    layer1
])

In [6]:
model.compile(
    loss = tf.nn.losses.mae,
    optimizer = tf.nn.optimizers.gradient_descent(5e-3),
)

In [7]:
%timeit model.fit(x, y, epochs = int(2e3))

Epoch 1|2000 
[=========================]    Loss: 4999371.50000     Metric: 4999371.50000
Epoch 2|2000 
[=========================]    Loss: 4999352.00000     Metric: 4999352.00000
Epoch 3|2000 
[=========================]    Loss: 4999331.50000     Metric: 4999331.50000
Epoch 4|2000 
[=========================]    Loss: 4999311.50000     Metric: 4999311.50000
Epoch 5|2000 
[=========================]    Loss: 4999290.50000     Metric: 4999290.50000
Epoch 6|2000 
[=========================]    Loss: 4999268.50000     Metric: 4999268.50000
Epoch 7|2000 
[=========================]    Loss: 4999246.50000     Metric: 4999246.50000
Epoch 8|2000 
[=========================]    Loss: 4999224.50000     Metric: 4999224.50000
Epoch 9|2000 
[=========================]    Loss: 4999202.00000     Metric: 4999202.00000
Epoch 10|2000 
[=========================]    Loss: 4999179.00000     Metric: 4999179.00000
Epoch 11|2000 
[=========================]    Loss: 4999155.50000     Metric: 4999155.500

In [8]:
vars(model)

{'args': (),
 'kwargs': {},
 'layers': [<tensorwrap.nn.layers.Dense at 0x7fd4a95b3370>,
 'trainable_variables': [],
 'loss_fn': <CompiledFunction of <function mae at 0x7fd3e65ad3f0>>,
 'optimizer': <tensorwrap.nn.optimizers.gradient_descent at 0x7fd4a95b11e0>,
 'metrics': <CompiledFunction of <function mae at 0x7fd3e65ad3f0>>}